In [ ]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [1]:
from minio import Minio
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, month, year,monotonically_increasing_id,row_number,concat,lit
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType,TimestampNTZType,StringType
from pyspark.sql.window import Window
from pyspark.conf import SparkConf
import requests
import json
import pandas as pd
from binance import Client
from datetime import datetime, date
import os
from dotenv import load_dotenv, dotenv_values
from binance.helpers import date_to_milliseconds, interval_to_milliseconds
from binance.exceptions import BinanceRequestException, BinanceAPIException
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import time
import logging
import psycopg2
load_dotenv()

True

In [2]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
MINIO_USER = os.getenv("MINIO_ROOT_USER")
MINIO_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
client_binance = Client(API_KEY, SECRET_KEY)

In [3]:
spark = SparkSession.builder \
    .appName("CryptoETL") \
    .config("spark.jars", "/Users/hamza/Desktop/projects/postgresql-42.7.5.jar") \
    .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_USER)#turn into access key in the future 
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_PASSWORD)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

25/04/23 16:11:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/23 16:12:03 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
#read data for 1 crypto
#schema check/validation need to do this, create spark df using schema struct
#implement transformations
#add unique id for crypto,price,time
#split into 3 tables
#upload to postgres db

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) #set to debug to capture all levels
if logger.hasHandlers():
    logger.handlers.clear()
logger.propagate = False

In [ ]:
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

In [5]:
cryptos = ['BTCUSDT','ETHUSDT','LTCUSDT','BNBUSDT','DOGEUSDT']
client_minio = Minio(
        "localhost:9000",  # Make sure you're using port 9000 for the S3 API
        #minio_url,
        access_key = MINIO_USER,
        secret_key = MINIO_PASSWORD,
        secure=False  # Disable SSL if you're not using SSL certificates
        )

In [8]:
objects = client_minio.list_objects("binancedata", prefix="BNBUSDT", recursive=True)
filenames = [obj.object_name for obj in objects]
filenames = [f for f in filenames if "_SUCCESS" not in f]

In [64]:
df_bnb = spark.read.parquet("s3a://binancedata/BNBUSDT/Monthly/year=2019/month=10/part-00001-12233d6e-b55f-4e5a-8285-808d25f4bf53.c000.snappy.parquet")
for file in filenames:
    df = spark.read.parquet(f"s3a://binancedata/{file}")
    df_bnb = df_bnb.union(df)
                            

In [ ]:
def data_cleaning():
    pass
    #drop duplicates
    #convert data types
    #drop null values

In [ ]:
def df_to_table():
    pass\
    #read from postgres
    #convert table to df and join with dataframe
    #check if id
    #convert dataframe to sql tables

In [ ]:
crypto_dict = [{"name":"Bitcoin","ticker":"BTC"},
               {"name":"Ethereum","ticker":"ETH"},
               {"name":"Litecoin","ticker":"LTC"},
               {"name":"Dogecoin","ticker":"DOGE"},
               {"name":"BNB","ticker":"BNB"}
              ]
crypto_df = spark.createDataFrame(crypto_dict)
#crypto_df = crypto_df.withColumn('id',monotonically_increasing_id())

In [4]:
'''crypto_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/crypto") \
    .option("dbtable", "crypto") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append")\
    .save()'''

'crypto_df.write     .format("jdbc")     .option("url", "jdbc:postgresql://localhost:5432/crypto")     .option("dbtable", "crypto")     .option("user", "postgres")     .option("password", "postgres")     .option("driver", "org.postgresql.Driver")     .mode("append")    .save()'

In [10]:
read_sql = "SELECT * FROM Crypto"

df_crypto = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/crypto") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("query", read_sql)\
    .option("driver", "org.postgresql.Driver")\
    .load()

In [12]:
df_crypto = df_crypto.withColumn("trading pair", concat(df_crypto.ticker, lit("USDT")))

In [24]:
df_crypto = df_crypto.filter(col("trading pair") == "BNBUSDT")

In [45]:
crypto_id = df_crypto.collect()[0]['id']

In [68]:
df_bnb = df_bnb.withColumn("crypto_id",lit(crypto_id)) 

In [72]:
df_bnb

DataFrame[datetime: timestamp, Open Price: string, Close Price: string, Volume: string, crypto_id: int]